In [ ]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'test-00000-of-00001 (3).parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,target
0,verified,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,[8],[8],text,"I have a flute, a piano, a trombone, four stov...",8
1,verified,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,[15],[15],text,"I have an apple, three bananas, a strawberry, ...",15
2,verified,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,[3],[3],text,"I have a fridge, a chair, and a microwave. How...",3
3,verified,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,[14],[14],text,"I have three blackberries, two strawberries, a...",14
4,verified,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,[5],[5],text,"I have a couch, an oven, two microwaves, and a...",5


In [3]:
df['platinum_prompt'][0]

'Answer the following question. \n\nI have a flute, a piano, a trombone, four stoves, a violin, an accordion, a clarinet, a drum, two lamps, and a trumpet. How many musical instruments do I have?\n\nThink step-by-step. Then, provide the final answer as a single integer in the format: "Answer: XXX".'

In [4]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\Read the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

<>:11: SyntaxWarning: invalid escape sequence '\R'
<>:11: SyntaxWarning: invalid escape sequence '\R'
/var/folders/pq/1f7x2wm974vgk0qcq8kybbdm0000gn/T/ipykernel_75060/704851011.py:11: SyntaxWarning: invalid escape sequence '\R'
  return f"{match.group(1)}{match.group(2)}\Read the question again: {match.group(2)}{match.group(3)}"


In [5]:
df['RE2'][0]

'Answer the following question. \n\nI have a flute, a piano, a trombone, four stoves, a violin, an accordion, a clarinet, a drum, two lamps, and a trumpet. How many musical instruments do I have?\\Read the question again: I have a flute, a piano, a trombone, four stoves, a violin, an accordion, a clarinet, a drum, two lamps, and a trumpet. How many musical instruments do I have?\n\nThink step-by-step. Then, provide the final answer as a single integer in the format: "Answer: XXX".'

In [6]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer as a single integer in the format)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single integer in the format)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single integer in the format)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single integer in the format)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single integer in the format)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single integer in the format)',
    "Read the question again first. ",
    regex=True
)

In [7]:
df['sum_no_cot'][0]

'Answer the following question. \n\nI have a flute, a piano, a trombone, four stoves, a violin, an accordion, a clarinet, a drum, two lamps, and a trumpet. How many musical instruments do I have?\n\nSummarize the question first. Then, provide the final answer as a single integer in the format: "Answer: XXX".'

In [8]:
df.to_parquet('object_counting_add_task.parquet')

In [9]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,target,RE2,sum,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,verified,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,[8],[8],text,"I have a flute, a piano, a trombone, four stov...",8,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...,Answer the following question. \n\nI have a fl...
1,verified,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,[15],[15],text,"I have an apple, three bananas, a strawberry, ...",15,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...,Answer the following question. \n\nI have an a...
2,verified,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,[3],[3],text,"I have a fridge, a chair, and a microwave. How...",3,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...,Answer the following question. \n\nI have a fr...
3,verified,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,[14],[14],text,"I have three blackberries, two strawberries, a...",14,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...,Answer the following question. \n\nI have thre...
4,verified,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,[5],[5],text,"I have a couch, an oven, two microwaves, and a...",5,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer the following question. \n\nI have a co...,Answer th